**Building code-agent with huggingfaces smolagents and exploring its capabilities**

**Installing the packages**

In [ ]:
!pip install -q requests gradio smolagents python-dotenv


**Importing packages and logging into huggingface**

In [ ]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

import os
import io
import IPython.display
from PIL import Image
import base64

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv() # read local .env file

from huggingface_hub import login

login()

In [109]:
import numpy as np
import pandas as pd

**Defining the model**

In [110]:
from smolagents import HfApiModel, CodeAgent


model = HfApiModel(
    "Qwen/Qwen2.5-72B-Instruct",
    provider="together", # Choose a specific inference provider
    max_tokens=4096,
    temperature=0.1
)

**Setting up the agent**

In [111]:
agent = CodeAgent(
    model=model,
    tools=[],
    max_steps=10,
    additional_authorized_imports=["pandas", "numpy"],
    verbosity_level=2
)
agent.logger.console.width=66

**Creating the sample data consisting of distance, time and speed**

In [ ]:
import pandas as pd

# Create sample data
data = {
    "distance_km": [10, 25, 50, 100, 5],
    "time_hr": [0.5, 1, 1.5, 2, 0.25],
}

# Calculate speed
data["speed_kmph"] = [d / t for d, t in zip(data["distance_km"], data["time_hr"])]

# Create DataFrame
df = pd.DataFrame(data)

print(df)


**Dropping the speed column**

In [ ]:
df = df.drop("speed_kmph", axis=1)
df.head()

**Defining the task and running the agent**

In [ ]:
task = """Here is a dataframe of different distance and time taken.
Could you give me a comparative table (as a dataframe) and calculate speed column for each of them?
"""
agent.logger.level = 1 # Lower verbosity level
result = agent.run(
    task,
    additional_args={"suppliers_data": df},
)

In [115]:
print(result)

   distance_km  time_hr  speed_kmph
0           10     0.50   20.000000
1           25     1.00   25.000000
2           50     1.50   33.333333
3          100     2.00   50.000000
4            5     0.25   20.000000


From the above it can be seen that agent was able to generate the code to calculate the speed, the values are correct.

**Creating another dataset**

In [ ]:
import pandas as pd

data = {
    "age": [25, 32, 45, 29, 60],
    "weight_kg": [70, 85, 95, 68, 72],
    "height_cm": [175, 180, 165, 160, 170],
    "waist_cm": [82, 90, 105, 78, 85],
    "hip_cm": [95, 100, 110, 92, 98],
    "systolic_bp": [120, 135, 145, 118, 130],
    "diastolic_bp": [80, 88, 92, 76, 85],
    "resting_heart_rate": [72, 80, 78, 70, 74],
    "cholesterol_mg_dl": [180, 220, 250, 170, 200],
    "glucose_mg_dl": [90, 105, 130, 85, 110]
}

df = pd.DataFrame(data)

# Calculate BMI = weight (kg) / (height (m))^2
df["bmi"] = df["weight_kg"] / ((df["height_cm"] / 100) ** 2)

df.head()


In [117]:
df = df.drop('bmi',axis = 1)

**Task isto analyse the dataset and create all important columns based on the existing ones( to test feature engineering)**

In [ ]:
task = """Here is a dataframe, analyse it properly.
Could you give the important feature engineering columns each of them and add it to the existing dataframe?
"""
agent.logger.level = 1 # Lower verbosity level
result = agent.run(
    task,
    additional_args={"suppliers_data": df},
)

The agent was able to successfully generate the code to generate new features

In [119]:
print(result)


   age  weight_kg  height_cm  waist_cm  hip_cm  systolic_bp  diastolic_bp  \
0   25         70        175        82      95          120            80   
1   32         85        180        90     100          135            88   
2   45         95        165       105     110          145            92   
3   29         68        160        78      92          118            76   
4   60         72        170        85      98          130            85   

   resting_heart_rate  cholesterol_mg_dl  glucose_mg_dl        BMI  \
0                  72                180             90  22.857143   
1                  80                220            105  26.234568   
2                  78                250            130  34.894399   
3                  70                170             85  26.562500   
4                  74                200            110  24.913495   

   Waist_to_Hip_Ratio  Blood_Pressure_Ratio  Cholesterol_to_Glucose_Ratio  \
0            0.863158              1.50